In [2]:
#!pip3 install fastai

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm

In [4]:
!ls

House-Prices-Sklearn.ipynb sample_submission.csv
House-Prices-Torch.ipynb   test.csv
my_submission.csv          train.csv
my_submission_nn.csv


In [5]:
def handle_df(df):
    
    data = pd.get_dummies(df, prefix_sep="__").fillna(0)
    data['YearBuilt'] = 2019 - data['YearBuilt']
    data['YearRemodAdd'] = 2019 - data['YearRemodAdd']
    
    return data

train = handle_df(pd.read_csv('train.csv'))
test = handle_df(pd.read_csv('test.csv'))

In [6]:
cat_dummies = [col for col in train 
               if "__" in col]

for col in cat_dummies:
    if col not in test.columns:
        print("Adding missing feature {}".format(col))
        test[col] = 0

test = test.reindex(train.columns, axis=1).drop(['SalePrice'], axis=1).fillna(0)

Adding missing feature Utilities__NoSeWa
Adding missing feature Condition2__RRAe
Adding missing feature Condition2__RRAn
Adding missing feature Condition2__RRNn
Adding missing feature HouseStyle__2.5Fin
Adding missing feature RoofMatl__ClyTile
Adding missing feature RoofMatl__Membran
Adding missing feature RoofMatl__Metal
Adding missing feature RoofMatl__Roll
Adding missing feature Exterior1st__ImStucc
Adding missing feature Exterior1st__Stone
Adding missing feature Exterior2nd__Other
Adding missing feature Heating__Floor
Adding missing feature Heating__OthW
Adding missing feature Electrical__Mix
Adding missing feature GarageQual__Ex
Adding missing feature PoolQC__Fa
Adding missing feature MiscFeature__TenC


In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(train, alpha=0.2, figsize=(6, 6), diagonal='kde')

In [7]:
train.sample(5)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType__ConLw,SaleType__New,SaleType__Oth,SaleType__WD,SaleCondition__Abnorml,SaleCondition__AdjLand,SaleCondition__Alloca,SaleCondition__Family,SaleCondition__Normal,SaleCondition__Partial
720,721,120,0.0,6563,8,5,34,34,0.0,1148,...,0,0,0,1,0,0,0,0,1,0
1339,1340,20,120.0,9560,5,7,47,47,0.0,360,...,0,0,0,1,0,0,0,0,1,0
1226,1227,60,86.0,14598,6,5,12,12,74.0,0,...,0,0,0,1,0,0,0,0,1,0
231,232,60,174.0,15138,8,5,24,23,506.0,689,...,0,0,0,1,0,0,0,0,1,0
1111,1112,60,80.0,10480,7,6,43,43,660.0,403,...,0,0,0,1,0,0,0,0,1,0


In [11]:
test.sample(5)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType__ConLw,SaleType__New,SaleType__Oth,SaleType__WD,SaleCondition__Abnorml,SaleCondition__AdjLand,SaleCondition__Alloca,SaleCondition__Family,SaleCondition__Normal,SaleCondition__Partial
1261,2722,60,72.0,8640,5,5,51,51,300.0,361.0,...,0,0,0,1,0,0,0,0,1,0
348,1809,70,90.0,9900,5,8,109,17,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0
964,2425,70,113.0,21281,5,4,84,12,0.0,0.0,...,0,0,0,1,0,0,0,1,0,0
1091,2552,20,64.0,6400,5,7,60,19,0.0,77.0,...,0,0,0,1,0,0,0,0,1,0
1066,2527,20,39.0,16300,5,4,42,42,0.0,60.0,...,0,0,0,1,0,0,0,0,1,0


In [12]:
# X_train = np.asarray(train[['LotArea', 'LotFrontage']]).astype(np.float32)
# y_train = np.asarray(train['SalePrice']).astype(np.float32)

#x_values = [i for i in range(11*2)]
X_train = np.array(np.asarray(train.drop(['SalePrice'], axis=1)), dtype=np.float32)

#y_values = [2*i + 1 for i in range(11)]
y_train = np.array(np.asarray(train['SalePrice'].fillna(0)), dtype=np.float32)


In [13]:
X_train

array([[1.000e+00, 6.000e+01, 6.500e+01, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [2.000e+00, 2.000e+01, 8.000e+01, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [3.000e+00, 6.000e+01, 6.800e+01, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       ...,
       [1.458e+03, 7.000e+01, 6.600e+01, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [1.459e+03, 2.000e+01, 6.800e+01, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [1.460e+03, 2.000e+01, 7.500e+01, ..., 0.000e+00, 1.000e+00,
        0.000e+00]], dtype=float32)

In [14]:
y_train

array([208500., 181500., 223500., ..., 266500., 142125., 147500.],
      dtype=float32)

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [16]:
reg = RandomForestRegressor(max_depth=100, n_estimators=1000, n_jobs=-1)
reg.fit(X_train, y_train)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=100,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

In [17]:
reg.score(X_train, y_train)

0.98188031513126

In [18]:
cross_val_score(reg, X_train, y_train, cv=3)

array([0.88679215, 0.85152342, 0.82359217])

In [164]:
# output = train.copy()
# output['SalePrice_pred'] = reg.predict(output.drop(['SalePrice'], axis=1))
# output[['SalePrice', 'SalePrice_pred']].sample(15)

In [169]:
output = test.copy()

output['SalePrice'] = reg.predict(output)
output[['Id', 'SalePrice']].to_csv('my_submission.csv', index=False)

In [166]:
!cat my_submission.csv

Id,SalePrice
1461,127963.168
1462,153952.38
1463,177029.696
1464,182570.994
1465,198721.046
1466,183685.152
1467,167688.5
1468,177032.762
1469,183336.858
1470,120354.054
1471,199068.644
1472,93631.186
1473,97336.3
1474,155737.548
1475,146576.886
1476,386518.788
1477,254750.222
1478,306191.962
1479,244103.202
1480,456016.718
1481,310757.474
1482,210239.648
1483,178531.596
1484,177978.964
1485,172806.594
1486,199459.722
1487,332212.436
1488,239684.12
1489,216064.946
1490,196485.024
1491,190876.558
1492,95342.206
1493,175718.766
1494,295428.182
1495,295479.22
1496,229062.666
1497,195051.242
1498,155099.908
1499,153919.918
1500,149817.282
1501,175084.134
1502,167222.926
1503,300980.094
1504,228986.958
1505,217847.064
1506,182779.816
1507,218786.092
1508,200954.004
1509,163474.452
1510,151669.892
1511,152318.782
1512,169557.666
1513,154029.038
1514,159410.35
1515,180863.188
1516,161223.792
1517,153054.988
1518,130435.228
1519,223946.232
1520,135737.218
1521,144712.218
1522,162569.548
1523,1